In [1]:
# Based on the example here: https://docs.bokeh.org/en/latest/docs/examples/topics/categorical/periodic.html
# This example does not include the Lanthanide and Actinide Series - here I have added them, and expanded some hover tools.

In [2]:
# !pip install bokeh==3.0.3

In [3]:
from bokeh.plotting import figure, show, column
from bokeh.sampledata.periodic_table import elements
from bokeh.transform import dodge, factor_cmap
import pandas as pd

In [4]:
from bokeh.io import output_notebook

In [5]:
output_notebook()

Loading BokehJS ...

In [6]:
save_to_file = False # Should we save this to a file, or display in the notebook?
if save_to_file:
    from bokeh.plotting import output_file, save
    output_file(filename="interactive_periodic_table.html", title="Interactive Periodic Table")
    command = save
else:
    command = show

In [7]:
# From Bokeh's example gallery
periods = ["I", "II", "III", "IV", "V", "VI", "VII"]
groups = [str(x) for x in range(1, 19)]

df = elements.copy()
df["atomic mass"] = df["atomic mass"].astype(str)
df["group"] = df["group"].astype(str)
df["period"] = [periods[x-1] for x in df.period]
df = df[df.group != "-"]
df = df[df.symbol != "Lr"]
df = df[df.symbol != "Lu"]

cmap = {
    "alkali metal"         : "#a6cee3",
    "alkaline earth metal" : "#1f78b4",
    "metal"                : "#d93b43",
    "halogen"              : "#999d9a",
    "metalloid"            : "#e08d49",
    "noble gas"            : "#eaeaea",
    "nonmetal"             : "#f1d4Af",
    "transition metal"     : "#599d7A",
    "lanthanoid"           : "#FFF88C",
    "actinoid"             : "#FF8CE1"
}

TOOLTIPS = [
    ("Name", "@name"),
    ("Atomic number", "@{atomic number}"),
    ("Atomic mass (amu) ", "@{atomic mass}"),
    ("Type", "@metal"),
    ("CPK color", "$color[hex, swatch]:CPK"),
    ("Electronic configuration", "@{electronic configuration}"),
    ('Electronegativity', "@electronegativity"),
    ('Atomic Radius (pm)', '@{atomic radius}'),
    ('Ion Radius (pm)', '@{ion radius}'), 
    ('VdW Radius (pm)', '@{van der Waals radius}'), 
    ('Standard State', '@{standard state}'),
    ('Bonding Type', '@{bonding type}'), 
    ('Melting Point (K)', '@{melting point}'), 
    ('Boiling Point (K)', '@{boiling point}'), 
    ('Density (g/m^3)', '@density')
]

p = figure(title="Periodic Table", width=1000, height=450,
           x_range=groups, y_range=list(reversed(periods)),
           tools="hover", toolbar_location=None, tooltips=TOOLTIPS)

r = p.rect("group", "period", 0.95, 0.95, source=df, fill_alpha=0.6, legend_field="metal",
           color=factor_cmap('metal', palette=list(cmap.values()), factors=list(cmap.keys())))

text_props = dict(source=df, text_align="left", text_baseline="middle")

x = dodge("group", -0.4, range=p.x_range)

p.text(x=x, y="period", text="symbol", text_font_style="bold", **text_props)

p.text(x=x, y=dodge("period", 0.3, range=p.y_range), text="atomic number",
       text_font_size="11px", **text_props)

p.text(x=x, y=dodge("period", -0.35, range=p.y_range), text="name",
       text_font_size="7px", **text_props)

p.text(x=x, y=dodge("period", -0.2, range=p.y_range), text="atomic mass",
       text_font_size="7px", **text_props)

p.rect(["3", "3"], ["VI", "VII"], 0.95, 0.95, color=["#FFF88C", "#FF8CE1"])

p.outline_line_color = None
p.grid.grid_line_color = None
p.axis.axis_line_color = None
p.axis.major_tick_line_color = None
p.axis.major_label_standoff = 0
p.legend.orientation = "horizontal"
p.legend.location ="top_center"
p.hover.renderers = [r] 

# Add La/Ac series
la_ac_series = elements.copy()

Lr = la_ac_series[la_ac_series.symbol == 'Lr']
Lr.group = "17"
Lr.metal = 'actinoid'
Lu = la_ac_series[la_ac_series.symbol == 'Lu']
Lu.group = "17"
Lu.metal = 'lanthanoid'

la_ac_series = la_ac_series[
    (la_ac_series.group == '-')
]

la_ac_series = pd.concat((la_ac_series, Lu, Lr))
la_ac_series = la_ac_series.sort_values(by='atomic number')

la_ac_series["group"] = [str(x) for x in [3+i for i in range(15)]*2]
la_ac_series["period"] = ["1" for i in range(15)] + ["2" for i in range(15)] 

p2 = figure(title="", width=1000, height=65*2,
           x_range=groups, 
           y_range=["2", "1"],
           tools="hover", toolbar_location=None, tooltips=TOOLTIPS)

la_ac_text_props = dict(source=la_ac_series, text_align="left", text_baseline="middle")

r2 = p2.rect("group", "period", 0.95, 0.95, source=la_ac_series, fill_alpha=0.6, 
           color=factor_cmap('metal', palette=list(cmap.values()), factors=list(cmap.keys())))

x = dodge("group", 
          -0.4, 
          range=p2.x_range)

p2.text(x=x, y="period", text="symbol", text_font_style="bold", **la_ac_text_props)

p2.text(x=x, y=dodge("period", 0.3, range=p2.y_range), text="atomic number",
       text_font_size="11px", **la_ac_text_props)

p2.text(x=x, y=dodge("period", -0.35, range=p2.y_range), text="name",
       text_font_size="7px", **la_ac_text_props)

p2.text(x=x, y=dodge("period", -0.2, range=p2.y_range), text="atomic mass",
       text_font_size="7px", **la_ac_text_props)

p2.outline_line_color = None
p2.grid.grid_line_color = None
p2.axis.axis_line_color = None
p2.axis.major_tick_line_color = None
p2.axis.major_label_standoff = 0
p2.xaxis.major_label_text_font_size = '0pt'  
p2.yaxis.major_label_text_font_size = '0pt'  

p2.hover.renderers = [r2]

command(column(p, p2))

/tmp/ipykernel_414187/3153946543.py:81: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Lr.group = "17"
/tmp/ipykernel_414187/3153946543.py:82: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Lr.metal = 'actinoid'
/tmp/ipykernel_414187/3153946543.py:84: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#retur

'/home/nam4/Desktop/interactive_periodic_table.html'